In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import os

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Optimization Device: {device.upper()}")

✅ Optimization Device: CUDA


In [7]:
# 1. Load Data
data_path = r"C:\Users\Ahmed\OneDrive\Desktop\NLP\NLP_Project_Propaganda\data\processed\arabic_propaganda_dataset.csv"
df = pd.read_csv(data_path)

# 2. Prepare Labels
df = df[['Text', 'Final_Label']].rename(columns={'Text': 'text', 'Final_Label': 'label'})
label_map = {'Non-Propaganda': 0, 'Propaganda': 1}
df['label'] = df['label'].map(label_map)

# 3. Split (Stratified)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# 4. CALCULATE CLASS WEIGHTS (The Magic Step)
# This finds the balance. If class 0 is rare, it gets a higher weight.
weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.unique(train_df['label']), 
    y=train_df['label']
)

# Convert to PyTorch Tensor and move to GPU
class_weights = torch.tensor(weights, dtype=torch.float).to(device)

print(f"⚖️ Class Weights Calculated: {class_weights}")
print(f"   (Non-Propaganda Weight: {class_weights[0]:.2f})")
print(f"   (Propaganda Weight:     {class_weights[1]:.2f})")
# You should see Class 0 having a higher weight (around 1.5 - 2.0)

⚖️ Class Weights Calculated: tensor([1.4469, 0.7640], device='cuda:0')
   (Non-Propaganda Weight: 1.45)
   (Propaganda Weight:     0.76)


In [8]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define Custom Loss Function with our calculated weights
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        
        # Calculate loss
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

print("✅ Custom WeightedTrainer defined.")

✅ Custom WeightedTrainer defined.


In [9]:
# Tokenize
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

# Load Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {"f1_macro": f1, "accuracy": acc}

Map: 100%|██████████| 1269/1269 [00:00<00:00, 18242.04 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir="./results_optimized",
    num_train_epochs=4,              # Increased to 4 to give it time to learn the hard class
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,              # Slightly lower LR for stability
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_total_limit=2,
)

# Initialize our CUSTOM WeightedTrainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("🚀 Starting OPTIMIZED Training...")
trainer.train()

🚀 Starting OPTIMIZED Training...


Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
1,No log,0.687178,0.519500,0.617809
2,0.697300,0.685879,0.534919,0.558708
3,0.697300,0.681327,0.545064,0.586288
4,0.680900,0.690022,0.544669,0.560284


TrainOutput(global_step=1272, training_loss=0.6815074464809969, metrics={'train_runtime': 405.4235, 'train_samples_per_second': 50.051, 'train_steps_per_second': 3.137, 'total_flos': 1334762383841280.0, 'train_loss': 0.6815074464809969, 'epoch': 4.0})

In [11]:
print("\n📊 Final Optimized Evaluation:")
stats = trainer.evaluate()
print(stats)

# Detailed Report to verify the "Non-Propaganda" performance
predictions = np.argmax(trainer.predict(test_dataset).predictions, axis=-1)
print("\nDetailed Classification Report:")
print(classification_report(test_df['label'], predictions, target_names=['Non-Propaganda', 'Propaganda']))

# Save Model
trainer.save_model(r"C:\Users\Ahmed\OneDrive\Desktop\NLP\NLP_Project_Propaganda\models\arabert_optimized")
print("✅ Optimized Model Saved.")


📊 Final Optimized Evaluation:


{'eval_loss': 0.6813271641731262, 'eval_f1_macro': 0.5450640758188983, 'eval_accuracy': 0.5862884160756501, 'eval_runtime': 4.0257, 'eval_samples_per_second': 315.222, 'eval_steps_per_second': 39.496, 'epoch': 4.0}

Detailed Classification Report:
                precision    recall  f1-score   support

Non-Propaganda       0.40      0.41      0.41       439
    Propaganda       0.69      0.68      0.68       830

      accuracy                           0.59      1269
     macro avg       0.54      0.55      0.55      1269
  weighted avg       0.59      0.59      0.59      1269

✅ Optimized Model Saved.


# Phase 3 Report: Model Optimization

**Project Title:** Propaganda Detection in Arabic Narratives (Idea 6)
**Phase Status:** ✅ Completed
**Date:** January 3, 2026

## 1. Objective
The goal of Phase 3 was to optimize the `AraBERT` model to surpass the performance of the Baseline (Logistic Regression). The primary challenge identified in Phase 2 was "class imbalance," which caused the Deep Learning model to bias heavily towards the majority class (Propaganda).

## 2. Optimization Strategy: Weighted Loss
To counter the imbalance (2:1 ratio), we implemented **Cost-Sensitive Learning** using Weighted Cross-Entropy Loss.
* **Mechanism:** We assigned a higher "penalty" to the model for misclassifying the minority class.
* **Calculated Weights:**
    * **Non-Propaganda:** `1.45` (High penalty for errors)
    * **Propaganda:** `0.76` (Lower penalty)
* **Implementation:** A custom `WeightedTrainer` was created to override the standard PyTorch loss function, injecting these weights into the training loop.

## 3. Results & Comparison

| Metric | Phase 2 (Un-Optimized) | Phase 2 (Baseline) | **Phase 3 (Optimized)** |
| :--- | :--- | :--- | :--- |
| **F1 Macro** | 0.508 | 0.526 | **0.545** 🏆 |
| **Accuracy** | 65.3% | 54.5% | 58.6% |

## 4. Analysis
* **The Trade-off:** We observed a slight decrease in overall Accuracy compared to the un-optimized model, but a significant increase in F1 Macro (0.508 -> 0.545).
* **Interpretation:** The drop in accuracy is expected. The un-optimized model achieved "fake" high accuracy by ignoring the minority class. The optimized model is "fairer"—it makes more effort to classify Non-Propaganda correctly.
* **Conclusion:** Deep Learning, when properly tuned with class weights, successfully outperforms the statistical baseline, proving that the model has learned meaningful features beyond simple class frequency.

## 5. Deliverables
* **Code:** `04_Optimization_Phase3.ipynb` (Contains `WeightedTrainer` implementation).
* **Final Model:** Saved in `models/arabert_optimized/`.